# Kaggle Project
## Linear Model

In [1]:
# Import required libraries
from __future__ import division
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt

np.random.seed(0)
%matplotlib inline

In [2]:
path_to_file = "/Users/justinchung/Documents/School/CS178/HW4/data"

# Training data
X = np.genfromtxt(path_to_file + '/X_train.txt', delimiter=None)
Y = np.genfromtxt(path_to_file + '/Y_train.txt', delimiter=None)
X,Y = ml.shuffleData(X,Y)

Xtr,Xva,Ytr,Yva = ml.splitData(X,Y); # split data into 80/20 train/validation
#Xtr, Ytr = ml.shuffleData(Xtr, Ytr)
Xtr, Ytr = Xtr[:4000], Ytr[:4000]

# Test data
Xte = np.genfromtxt(path_to_file + '/X_test.txt', delimiter=None)

In [3]:
# Rescale training and test data
Xtr,args = ml.transforms.rescale(Xtr)
Xte,_ = ml.transforms.rescale(Xtr, args)

In [7]:
XtrP = ml.transforms.fpoly(Xtr, 2, bias=False)

XtrP,argsP = ml.transforms.rescale(XtrP)
XteP,_ = ml.transforms.rescale(XtrP, argsP)
    
learner = ml.linearC.linearClassify()
learner.train(XtrP, Ytr, reg=0.1, initStep=1.0, stopTol=1e-4, stopIter=100)
    
probs = learner.predictSoft(XteP)
print(probs)
print(str(learner.auc(XtrP, Ytr)))

[[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 ..., 
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]]
0.509360192372


In [ ]:
probs = learner.predictSoft(Xte)
print(probs)

print(learner.auc(Xtr, Ytr))